In [39]:
#import statements
import pandas as pd
import Levenshtein
import re
import numpy as np
from functools import reduce
import math

#get files from computer
path='/Users/sidsharma/Downloads/Master 50 State Data (Rev 2.0 for 2018-19) - Pennsylvania (1).csv'
path2 ='/Users/sidsharma/Downloads/Cleaned - Master 50 State Data - Pennsylvania.csv'
pd.set_option('display.float_format', lambda x: '%.1f' % x)


#read csv and fill in missing data
Mass1819 = pd.read_csv(path)
Mass1819.rename(columns = {'School': 'School Name'}, inplace = True)
print(Mass1819.columns)
Mass1819['Updated in'] = "2018"
Mass1718 = pd.read_csv(path2)
Mass1718.rename(columns = {'School': 'School Name'}, inplace = True)
Mass1718['Updated in'] = "2017"
Mass1718 = Mass1718.applymap(lambda x: x.strip() if isinstance(x, str) else x)
Mass1819 = Mass1819.applymap(lambda x: x.strip() if isinstance(x, str) else x)

unique1 = Mass1819.dropna(subset = ['NCES School ID']).copy(deep = True)
dict1 = unique1.set_index('School Name').to_dict()['NCES School ID']

def replace_NCES(df):
    if isinstance(df['NCES School ID'], str):
        return df['NCES School ID']
    else:
        if (math.isnan(df['NCES School ID'])):
            NCES = dict1.get(df['School Name'], None)
            return NCES
        else:
            return df['NCES School ID']

Mass1819['NCES School ID'] = Mass1819.apply(replace_NCES, axis = 1)
Mass1718['NCES School ID'] = Mass1718.apply(replace_NCES, axis = 1)


#combine, sort, and make everything lowercase
df = Mass1819.append(Mass1718, sort = False)
df.sort_values('School Name', inplace = True)
df.drop(columns = ['Requirement', 'Notes'], inplace = True)
df.rename(columns = lambda x: x.strip(), inplace = True)

#change out states
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

df['State'].replace(states, inplace = True)

#combine, sort, and make everything lowercase
state = df.loc[1]['State'].iloc[0]
df['Class Name'] = df['Class Name'].apply(lambda x: x.lower() if isinstance(x,str) else x)
temp = ''
df2 = pd.DataFrame()

#check if a string has either a digit or a roman numeral
def contains(item):
    if re.search('[0-9]', item):
        return True
    elif item.find('ii') != -1:
        return True
    return False

#iterate through dataframe
for i in df['School Name']:
    #check if we already iterated through this one
    if temp == i:
        continue
        
    #if not already done, divide dataframe for specific school into 2018 and 2017
    else:
        temp = i
        tempdf18 = df[(df['School Name'] == i) & (df['Updated in'] == "2018")]
        tempdf17 = df[(df['School Name'] == i) & (df['Updated in'] == "2017")]   
        tempdf18.reset_index(inplace = True)
        tempdf17.reset_index(inplace = True)
        
        #if there is at least 2 schools in each dataframe, iterate through
        if (len(tempdf18) > 1) & (len(tempdf17) > 1):
            for z in range(0, len(tempdf18)):
                class18name = tempdf18.loc[z]['Class Name']
                class18descript = tempdf18.loc[z]['Description of course']
                for w in range(0, len(tempdf17)):
                    class17name = tempdf17.loc[w]['Class Name']
                    class17descript = tempdf17.loc[w]['Description of course']
                    flag = False
                    
                    #run Levenshtein score on name and description of class
                    try:
                        if ((contains(class18name) == False) and (contains(class17name) == False)):
                            if (Levenshtein.distance(class18name,class17name)  <= 5 and 
                                Levenshtein.distance(class18name,class17name)  > 0):
                                #print(class18name, class17name, temp)
                                tempdf17.replace({class17name:class18name}, inplace = True)
                                flag = True
                        if (Levenshtein.distance(class18descript,class17descript)  <= 30 and 
                            Levenshtein.distance(class18descript,class17descript)  > 0 and 
                            Levenshtein.distance(class18name,class17name) != 0 and
                            flag == False):
                                #print("description-based", class18name, class17name)
                                tempdf18.replace({class18name:class17name}, inplace = True)
                    except(TypeError):
                        break
                        
        #concatenate cleaned and combined data into one dataframe                        
        df2 = pd.concat([df2, tempdf18, tempdf17])

df2.reset_index(inplace = True)


#keep good track for further summation
df2['Class Name Count'] = 1

#aggregation functions to combine various series of data
def testaddsemicolon(series):
    return reduce(lambda x, y: str(x) + '; ' + str(y), series)

def testaddcomma(series):
    return reduce(lambda x, y: str(x) + ', ' + str(y), series)

def lenseries(series):
    return len(series)

#recapitalize
df2['Class Name'] = df2['Class Name'].apply(lambda x: x.title() if isinstance(x, str) else x)
tempsheet = df2.copy(deep = True)
tempsheet = tempsheet[tempsheet['Updated in'] != '2017']

#determine standard
def standard(df):
    try: 
        temp = float(df['Duration'])
    except ValueError:
        temp = .5
    if (math.isnan(temp)):
        temp = .5
    if (temp>=.5 and (df['Grad'] == 'R' or df['Grad'] == 'Required') & (df['Type'] == 'S' or df['Type'] == 'Standalone')):
         return "Gold Standard"
    elif (temp>=.5 and (df['Grad'] == 'E' or df['Grad'] == 'Cluster') & (df['Type'] == 'S' or df['Type'] == 'Standalone')):
        return "Silver Standard"
    elif ((df['Grad'] == 'E' or df['Grad'] == 'Cluster') or (df['Grad'] == 'R' or df['Grad'] == 'Required')):
        return "Bronze Standard"
    else: 
        return "Incomplete Information"
    
tempsheet['Standard'] = tempsheet[['Duration', 'Grad', 'Type', 'Class Name']].apply(standard, axis = 1)
print(df2.columns)
#group by and aggregate
df2 = df2.groupby(['NCES School ID', 'School Name', 'Class Name']).agg(({'Class Name Count': lenseries, 'Description of course': testaddsemicolon, 
                                                                  'Year of Course Catalog': testaddcomma, 'Updated in': testaddcomma,
                                                                  'Link to School Website': lambda x: x.iloc[0] , 'Link to Course Catalog': lambda x: x.iloc[0]}))
df2 = df2[df2['Updated in'] != "2017"]
df2.rename({'Class Name Count': 'If 2, was offered last year'}, axis = 'index', inplace = True)
keep = ['NCES School ID','Street Address', 'Students*', 'School Name', 
       'Link to School Website', 'Link to Course Catalog',
       'Source', 'Class Name', 'Description of course', 'Duration', 'Grad',
       'Type', 'Year of Course Catalog', 'Class Name Count', 'Standard']
tempsheet = tempsheet[keep]
tempsheet = tempsheet.groupby(['NCES School ID', 'School Name']).agg(({'Class Name': testaddcomma, 'Class Name Count': np.sum,
                                                           'Link to Course Catalog': lambda x: x.iloc[0], 
                                                           'Description of course': testaddsemicolon,
                                                           'Grad': testaddcomma, 'Type': testaddcomma,
                                                           'Standard': testaddcomma}))
tempsheet.reset_index(inplace = True)

def highest_level(item):
    if isinstance(item, str):                                                            
        if ((item.find('S') != -1) or (item.find('Standalone') != -1)):
            return 'Standalone'
        elif (item.find('E') != -1 or (item.find('Standalone') != -1)):
            return 'Umbrella'
    return 'No Access'                                                             

def highest_standard(item):
    if isinstance(item, str):
        if item.find('Gold Standard') != -1:
            return 'Gold Standard'
        elif item.find('Silver Standard') != -1:
            return 'Silver Standard'
        elif item.find('At least Bronze Standard') != -1:
            return 'At least Bronze Standard'
        elif item.find('Bronze Standard') != -1:
            return 'Bronze Standard'
    return 'Incomplete Information'
                                

tempsheet['Highest Level'] = tempsheet['Type'].apply(highest_level)
tempsheet['Highest Standard'] = tempsheet['Standard'].apply(highest_standard)


#process NCES file
ELSI = pd.read_csv('ELSI_csv_export_6368743319348959414332.csv', skiprows = 6)
ELSI.replace('†', np.nan, inplace = True)
ELSI.replace('="0"', np.nan, inplace = True)
ELSI.replace('–', np.nan, inplace = True)

def replace(item):
    if item.startswith('="'):
        return item[2:-1]
    else:
        return item


#clean up column names
def clean(item):
    temp = item.find('[Public School]')
    if temp != -1:
        return item[:temp-1]
    else:
        return item

#clean up file
ELSI.rename(columns = clean, inplace = True)
ELSI['State Name'] = ELSI['State Name'].apply(lambda x: x.title() if isinstance(x, str) else x)
ELSI = ELSI[ELSI['State Name'] == state]
ELSI['School ID - NCES Assigned'] = ELSI['School ID - NCES Assigned'].apply(replace)
tempsheet['NCES School ID'] = tempsheet['NCES School ID'].convert_objects(convert_numeric=True)
tempsheet['NCES School ID'] = tempsheet['NCES School ID'].astype('float64')
ELSI['School ID - NCES Assigned'] = ELSI['School ID - NCES Assigned'].astype('float64')
ELSI['Black Students'] = ELSI['Black Students'].astype('float64')
ELSI['Hispanic Students'] = ELSI['Hispanic Students'].astype('float64')
ELSI['Free Lunch Eligible'] = ELSI['Free Lunch Eligible'].astype('float64')
ELSI['Grades 9-12 Students'] = ELSI['Grades 9-12 Students'].astype('float64')
ELSI['Total Students All Grades (Excludes AE)'] = ELSI['Total Students All Grades (Excludes AE)'].astype('float64')
tempsheet = tempsheet.merge(ELSI, left_on = 'NCES School ID', right_on = 'School ID - NCES Assigned', how = 'left')
#summary statistics
print(state)
print('Grades 9-12 checked', tempsheet['Grades 9-12 Students'].sum())
print('Grades 9-12 total', ELSI['Grades 9-12 Students'].sum())

gold = tempsheet[tempsheet['Highest Standard'] == 'Gold Standard']
silver = tempsheet[tempsheet['Highest Standard'] == 'Silver Standard']
bronze = tempsheet[tempsheet['Highest Standard'] == 'Bronze Standard']
at_least = tempsheet[tempsheet['Highest Standard'] == 'At least Bronze Standard']

print('Grades 9-12 covered', gold['Grades 9-12 Students'].sum() + silver['Grades 9-12 Students'].sum() + bronze['Grades 9-12 Students'].sum() + at_least['Grades 9-12 Students'].sum())
print('Gold Standard 9-12', gold['Grades 9-12 Students'].sum())
print('Percent black covered', round(tempsheet['Black Students'].sum()/tempsheet['Total Students All Grades (Excludes AE)'].sum(), 4))
print('Percent black covered (Gold Standard/Silver Standard)', round((gold['Black Students'].sum() + silver['Black Students'].sum())/(gold['Total Students All Grades (Excludes AE)'].sum() + silver['Total Students All Grades (Excludes AE)'].sum()), 4))
print('Percent hispanic covered', round(tempsheet['Hispanic Students'].sum()/tempsheet['Total Students All Grades (Excludes AE)'].sum(), 4))
print('Percent hispanic covered (Gold Standard/Silver Standard)', round((gold['Hispanic Students'].sum() + silver['Hispanic Students'].sum())/(gold['Total Students All Grades (Excludes AE)'].sum() + silver['Total Students All Grades (Excludes AE)'].sum()), 4))
print('Percent Free Lunch covered', round(tempsheet['Free Lunch Eligible'].sum()/tempsheet['Total Students All Grades (Excludes AE)'].sum(), 4))
print('Percent Free Lunch covered (Gold Standard/Silver Standard)', round((gold['Free Lunch Eligible'].sum() + silver['Free Lunch Eligible'].sum())/(gold['Total Students All Grades (Excludes AE)'].sum() + silver['Total Students All Grades (Excludes AE)'].sum()), 4))
print('Charters covered', tempsheet['Charter School'].value_counts())
print('Charters covered (Gold Standard)', gold['Charter School'].value_counts())

standalone = tempsheet[tempsheet['Highest Level'] == 'Standalone']
print('Standalone Schools', len(standalone))
print('Standalone Students 9-12 Students', standalone['Grades 9-12 Students'].sum())

#print('Gold Standard Schools', len(gold))
print('Covered Schools', len(gold) + len(silver) + len(bronze)+ len(at_least))
print('Checked Schools', len(tempsheet))
print('Total Schools', len(ELSI))
print('Percent unique', round(len(df2[df2['Class Name Count'] == 1])/len(df2), 4))
print(tempsheet['Highest Standard'].value_counts())
print(len(tempsheet))


df2.reset_index(inplace = True)
print(len(df2))
df2['NCES School ID'] = df2['NCES School ID'].convert_objects(convert_numeric=True)
print(len(df2))
df2['NCES School ID'] = df2['NCES School ID'].astype('float64')
df2 = df2.merge(tempsheet, left_on = 'NCES School ID', right_on = 'NCES School ID', how = 'left')
keep = ['School Name_x', 'NCES School ID', 'Class Name_x', 'Class Name Count_x', 'Description of course_x', 'Total Students All Grades (Excludes AE)', 'Grades 9-12 Students', 'Full-Time Equivalent (FTE) Teachers', 'Highest Standard', 'Location Address 1', 'Location City',
       'Location State Abbr', 'Location ZIP', 'County Name','Latitude', 'Longitude', 'Year of Course Catalog', 'Link to School Website', 'Link to Course Catalog_x']
df2 = df2[keep]
df2 = df2[df2['Class Name_x'] != 'N/A']
df2 = df2[df2['Class Name_x'] != 'NA']
print(len(df2))

#send back to computer
path = '/Users/sidsharma/Downloads/CourseMasterList/'
path += state + ' - Course Master List.csv'
df2.to_csv(path)
            

Index(['NCES School ID', 'Low Grade*', 'High Grade*', 'School Name',
       'District', 'County Name*', 'Street Address', 'City', 'State', 'ZIP',
       'Phone', 'Locale*', 'Charter', 'Magnet*', 'Title I School*',
       'Students*', 'Teachers*', 'Free Lunch*', 'Reduced Lunch*',
       'Link to School Website', 'Link to Course Catalog', 'Source',
       'Year of Course Catalog', 'Status', 'Class Name', 'Requirement',
       'Description of course', 'Notes', 'Duration', 'Grad', 'Type', 'Stamp'],
      dtype='object')
Index(['level_0', 'index', 'NCES School ID', 'Low Grade*', 'High Grade*',
       'School Name', 'District', 'County Name*', 'Street Address', 'City',
       'State', 'ZIP', 'Phone', 'Locale*', 'Charter', 'Magnet*',
       'Title I School*', 'Students*', 'Teachers*', 'Free Lunch*',
       'Reduced Lunch*', 'Link to School Website', 'Link to Course Catalog',
       'Source', 'Year of Course Catalog', 'Status', 'Class Name',
       'Description of course', 'Duration', 'Grad', 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:279: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:324: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
